In [1]:
import sympy as sp
from IPython.display import display, Math
import math as m
import time
start_time = time.time()

In [2]:
# x, y sú súradnice bodov v R^2
x, y = sp.symbols('x y')
# t je parameter z intervalu I
t = sp.Symbol('t')
# a, b sú konštanty pre škálovanie elipsoidu, a - dotykový smer, b - normálový smer
a, b = sp.symbols('a b')
# všetky ostatné symboly sú parametre v parametrizácii krivky parametric_curve
c, d = sp.symbols('c d')

In [3]:
# Vstupné parametre
# interval vyčíslenia krivky
t_min = -10
t_max = 10
# parametre škálovania
a_0 = 2
b_0 = 1
# parametre parametrizácie
c_0 = 2
d_0 = 1

# parametrizácia krivky
parametric_curve = sp.Matrix([t, t**2]) 
#curve_name = 'parabola'

In [4]:
# Vypočíta dotyčnicu a normálu pre parametric_curve
def local_coordinate_system(parametric_curve): 
    t = sp.Symbol('t') 
    parametric_curve_first_derivation = sp.diff(parametric_curve, t)
    T = parametric_curve_first_derivation / sp.sqrt(parametric_curve_first_derivation.dot(parametric_curve_first_derivation))
    T = sp.simplify(T)
    N = sp.Matrix([-T[1], T[0]])
    N = sp.simplify(N)
    return T, N

# Vyrobí inverznú maticu prechodu 
def transition_matrix(T, N):
    P = sp.Matrix.hstack(T, N)
    return P.T

def matrix_of_quadrics(A, B, C, D, E, F):
    row1 = [A, B/2, D/2]
    row2 = [B/2, C, E/2]
    row3 = [D/2, E/2, F]
    matrix = sp.Matrix([row1, row2, row3])
    return matrix

def matrix_of_homogenious(A, B, C):
    row1 = [A, B/2]
    row2 = [B/2, C]
    matrix = sp.Matrix([row1, row2])
    return matrix

In [5]:
# Výpočet vektorov
T, N = local_coordinate_system(parametric_curve)
# Zostavenie matice
P = transition_matrix(T, N)
display(Math(sp.latex(P)))
# Maticou P vynásobíme vektor (X-parametric_curve) a tak zmeníme bázu elipsy
vector = sp.Matrix([x - parametric_curve[0], y - parametric_curve[1]])
result_vector = P * vector
display(Math(sp.latex(result_vector)))
ellipse_in_new_basis = result_vector[0]**2/a**2 + result_vector[1]**2/b**2 - 1
# Derivácia jednoparametrického systému elíps 
derivation = sp.diff(ellipse_in_new_basis, t)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
# Výpočet, o aký typ kužeľosečky ide
# Najprv potrebujeme extrahovať koeficienty z derivácie
expanded_expression = sp.expand(derivation)
A = expanded_expression.coeff(x**2)
B = expanded_expression.coeff(x*y)
C = expanded_expression.coeff(y**2)

# Tu bolo potrebné odstrániť všetky koeficienty pri členoch x*y, x*z,..
coef_x = expanded_expression.coeff(x)
terms_x = sp.Add(*coef_x.args) if isinstance(coef_x, sp.Add) else coef_x
terms_x_filtered = [term for term in sp.Add.make_args(terms_x) if not term.has(y)]
D = sp.Add(*terms_x_filtered)

coef_y = expanded_expression.coeff(y)
terms_y = sp.Add(*coef_y.args) if isinstance(coef_y, sp.Add) else coef_y
terms_y_filtered = [term for term in sp.Add.make_args(terms_y) if not term.has(x)]
E = sp.Add(*terms_y_filtered)

# Tu bolo potrebné odstrániť všetky koeficienty pri členoch x, y, z
terms = sp.Add(*expanded_expression.args) if isinstance(expanded_expression, sp.Add) else expanded_expression
terms_filtered = [term for term in sp.Add.make_args(terms) if not term.has(x) and not term.has(y)]
F = sp.Add(*terms_filtered)

matrix_of_derivation = matrix_of_quadrics(A, B, C, D, E, F)
submatrix = matrix_of_homogenious(A, B, C)
display(Math(sp.latex(submatrix)))

# Print koeficientov
print("Coefficient of x^2:")
display(Math(sp.latex(sp.simplify(A))))
print("Coefficient of y^2:")
display(Math(sp.latex(sp.simplify(C))))
print("Coefficient of xy:") 
display(Math(sp.latex(sp.simplify(B))))
print("Coefficient of x:")
display(Math(sp.latex(sp.simplify(D))))
print("Coefficient of y:")
display(Math(sp.latex(sp.simplify(E))))
print("Coefficient of constant term:")
display(Math(sp.latex(sp.simplify(F))))

<IPython.core.display.Math object>

Coefficient of x^2:


<IPython.core.display.Math object>

Coefficient of y^2:


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

Coefficient of y:


<IPython.core.display.Math object>

Coefficient of constant term:


<IPython.core.display.Math object>

In [7]:
# Typ kužeľosečky
determinant = sp.det(matrix_of_derivation)
subdeterminant = A*C-B**2/4

In [16]:
# Diagonalize the matrix
M = matrix_of_derivation.subs({a: 2, b: 1})
M1 = submatrix
P, D = M.diagonalize()

P1, D1 = M1.diagonalize()
subdet = D1.det()
display(Math(sp.latex(subdet)))
#D = M.eigenvals()
#P = M.eigenvects()

# Print the diagonalized matrix
#print("Diagonal matrix D:")
display(Math(sp.latex(D)))
#print("Eigenvector matrix P:")
display(Math(sp.latex(P)))

display(Math(sp.latex(D1)))
display(Math(sp.latex(P1)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
# smerové vektory hlavných osí
# Define eigenvectors (replace with your expressions)
eigen_vector0 = sp.sqrt(sp.Abs(D[0, 0])) * P[:, 0]
eigen_vector1 = sp.sqrt(sp.Abs(D[1, 1])) * P[:, 1]
eigen_vector2 = sp.sqrt(sp.Abs(D[2, 2])) * P[:, 2]
#display(Math(sp.latex(eigen_vector1)))
#display(Math(sp.latex(eigen_vector2)))

# Define a tolerance value for considering a number as zero
tolerance = 1e-15  # You can adjust this tolerance as needed

# Check if each eigenvector is effectively zero
is_nonzero0 = abs(D[0,0]) >= tolerance 
is_nonzero1 = abs(D[1,1]) >= tolerance 
is_nonzero2 = abs(D[2,2]) >= tolerance

# Determine which eigenvectors are nonzero
nonzero_eigenvalues = []
nonzero_eigenvectors = []
if is_nonzero0:
    nonzero_eigenvalues.append(D[0,0])
    #nonzero_eigenvectors.append(eigen_vector0/sp.sqrt(eigen_vector0[0]**2 + eigen_vector0[1]**2 + eigen_vector0[2]**2))
    nonzero_eigenvectors.append(eigen_vector0)
if is_nonzero1:
    nonzero_eigenvalues.append(D[1,1])
    nonzero_eigenvectors.append(eigen_vector1)
    #nonzero_eigenvectors.append(eigen_vector1/sp.sqrt(eigen_vector1[0]**2 + eigen_vector1[1]**2 + eigen_vector1[2]**2))
if is_nonzero2:
    nonzero_eigenvalues.append(D[2,2])
    nonzero_eigenvectors.append(eigen_vector2)
    #nonzero_eigenvectors.append(eigen_vector2/sp.sqrt(eigen_vector2[0]**2 + eigen_vector2[1]**2 + eigen_vector2[2]**2))

#print(len(nonzero_eigenvectors))

# po sčítaní osí dostávam žiadaný smer
Q1 = (nonzero_eigenvectors[0] + nonzero_eigenvectors[1])
if (nonzero_eigenvalues[1] >= nonzero_eigenvalues[0]):
    Q2 =(nonzero_eigenvectors[1] - nonzero_eigenvectors[0])
else:
    Q2 = (nonzero_eigenvectors[0] - nonzero_eigenvectors[1])

g = Q1[0]*x+Q1[1]*y+Q1[2]
h = Q2[0]*x+Q2[1]*y+Q2[2]
print(sp.latex(g) + ' = 0')
print(sp.latex(h) + ' = 0')

theta1 = m.atan2(Q1[1],Q1[0])
d1 = Q1[2]/ m.sqrt(Q1[0]**2+Q1[1]**2)
theta2 = m.atan2(Q2[1],Q2[0])
d2 = Q2[2]/ m.sqrt(Q2[0]**2+Q2[1]**2)

lineg = sp.cos(theta1)*x + sp.sin(theta1)*y + d1
lineh = sp.cos(theta2)*x + sp.sin(theta2)*y + d2

print(sp.latex(lineg))
print(sp.latex(lineh))

x \left(\left(- \frac{51}{91} + \frac{\sqrt{326}}{91}\right) \sqrt{\frac{93}{100} + \frac{7 \sqrt{326}}{100}} + \sqrt{- \frac{93}{100} + \frac{7 \sqrt{326}}{100}} \left(- \frac{51}{91} - \frac{\sqrt{326}}{91}\right)\right) + y \left(\sqrt{\frac{93}{100} + \frac{7 \sqrt{326}}{100}} \cdot \left(\frac{270}{91} - \frac{16 \sqrt{326}}{91}\right) + \sqrt{- \frac{93}{100} + \frac{7 \sqrt{326}}{100}} \cdot \left(\frac{270}{91} + \frac{16 \sqrt{326}}{91}\right)\right) + \sqrt{- \frac{93}{100} + \frac{7 \sqrt{326}}{100}} + \sqrt{\frac{93}{100} + \frac{7 \sqrt{326}}{100}} = 0
x \left(\left(- \frac{51}{91} + \frac{\sqrt{326}}{91}\right) \sqrt{\frac{93}{100} + \frac{7 \sqrt{326}}{100}} - \sqrt{- \frac{93}{100} + \frac{7 \sqrt{326}}{100}} \left(- \frac{51}{91} - \frac{\sqrt{326}}{91}\right)\right) + y \left(- \sqrt{- \frac{93}{100} + \frac{7 \sqrt{326}}{100}} \cdot \left(\frac{270}{91} + \frac{16 \sqrt{326}}{91}\right) + \sqrt{\frac{93}{100} + \frac{7 \sqrt{326}}{100}} \cdot \left(\frac{270}{91} - \

In [10]:
# Ukončenie merania času
end_time = time.time()
elapsed_time = end_time - start_time

In [11]:
# Typ kuželosečky vieme jasne dourčiť pre špecifikáciu parametrov
t_values = range(t_min, t_max)
is_negative = True

for t_val in t_values:
    if subdeterminant.subs({t: t_val, a: a_0, b: b_0, c: c_0, d: d_0}) >= 0:
        is_negative = False
        break

if is_negative:
    print("Výraz je pre zadané hodnoty záporný. ")
else:
    print("Výraz nie je pre zadané hodnoty záporný. ")

Výraz je pre zadané hodnoty záporný. 


In [12]:
print(parametric_curve)
print("Systém: ")
system = sp.latex(sp.simplify(ellipse_in_new_basis))
display(Math(system))
#print(system + '=0')

print("Derivácia sytému: ")
der = sp.latex(sp.simplify(derivation))
display(Math(der))
#print(der + '=0')

print("Determinant derivácie: ")
display(Math(sp.latex(sp.simplify(determinant))))

print("Čas: ")
print(elapsed_time, "seconds")

expression = sp.latex(sp.simplify(subdeterminant))
display(Math(expression))
if is_negative:
    print("Výraz je pre zadané hodnoty záporný. ")
else:
    print("Výraz nie je pre zadané hodnoty záporný. ")

Matrix([[t], [t**2]])
Systém: 


<IPython.core.display.Math object>

Derivácia sytému: 


<IPython.core.display.Math object>

Determinant derivácie: 


<IPython.core.display.Math object>

Čas: 
2.349496364593506 seconds


<IPython.core.display.Math object>

Výraz je pre zadané hodnoty záporný. 
